In [53]:
from read_logs import row_iterator
import datetime
import polars as pl
import matplotlib.pyplot as plt

In [49]:
rows = row_iterator("/Users/arc/iavl-bench-data/run-mixed-large2/memiavl-100-2.jsonl")
memiavl_rows = [r for r in rows if r.get("module") == "memiavl"]

In [50]:
print([[r.get("msg"), r.get("version")] for r in memiavl_rows if r.get("msg")][0:20])

[['start rewriting snapshot', 100], ['finished rewriting snapshot', 100], ['finished best-effort WAL catchup', 100], ['switched to new snapshot', 112], ['failed to find first snapshot', None], ['start rewriting snapshot', 200], ['finished rewriting snapshot', 200], ['finished best-effort WAL catchup', 200], ['switched to new snapshot', 221], ['failed to find first snapshot', None], ['start rewriting snapshot', 300], ['finished rewriting snapshot', 300], ['finished best-effort WAL catchup', 300], ['switched to new snapshot', 326], ['prune snapshot', None], ['start rewriting snapshot', 400], ['finished rewriting snapshot', 400], ['finished best-effort WAL catchup', 400], ['switched to new snapshot', 429], ['prune snapshot', None]]


In [51]:
snapshot_start = {}
snapshot_end = {}
snapshot_switch = {}
finish_wal_catchup = {}
snapshots = []
for r in memiavl_rows:
    msg = r.get("msg")
    time = datetime.datetime.fromisoformat(r.get("time"))
    version = r.get("version")
    match msg:
        case "start rewriting snapshot":
            snapshot_start[version] = time
            print(f"{time} SNAPSHOT START {version}")
            snapshots += [{"version": version, "start": time}]
        case "finished rewriting snapshot":
            snapshot_end[version] = time
            elapsed = time - snapshot_start[version]
            print(f"{time} SNAPSHOT END {version} {elapsed} ELAPSED")
            snapshots[-1]["end"] = time
        case "finished best-effort WAL catchup":
            finish_wal_catchup[version] = time
            elapsed = time - snapshot_start[version]
            print(f"{time} FINISH BEST-EFFORT WAL CATCHUP {version} {elapsed} ELAPSED")
            snapshots[-1]["wal_catchup"] = time
        case "switched to new snapshot":
            snapshot_switch[version] = time
            print(f"{time} SNAPSHOT SWITCH {version}")
            snapshots[-1]["switch_time"] = time
            snapshots[-1]["switch_version"] = version

snapshots = pl.DataFrame(snapshots)

2025-09-09 17:58:56.671489+00:00 SNAPSHOT START 100
2025-09-09 17:58:59.621843+00:00 SNAPSHOT END 100 0:00:02.950354 ELAPSED
2025-09-09 17:59:01.156508+00:00 FINISH BEST-EFFORT WAL CATCHUP 100 0:00:04.485019 ELAPSED
2025-09-09 17:59:02.687640+00:00 SNAPSHOT SWITCH 112
2025-09-09 17:59:28.413426+00:00 SNAPSHOT START 200
2025-09-09 17:59:33.319810+00:00 SNAPSHOT END 200 0:00:04.906384 ELAPSED
2025-09-09 17:59:36.012194+00:00 FINISH BEST-EFFORT WAL CATCHUP 200 0:00:07.598768 ELAPSED
2025-09-09 17:59:38.728401+00:00 SNAPSHOT SWITCH 221
2025-09-09 18:00:02.983921+00:00 SNAPSHOT START 300
2025-09-09 18:00:10.125621+00:00 SNAPSHOT END 300 0:00:07.141700 ELAPSED
2025-09-09 18:00:13.589576+00:00 FINISH BEST-EFFORT WAL CATCHUP 300 0:00:10.605655 ELAPSED
2025-09-09 18:00:16.868921+00:00 SNAPSHOT SWITCH 326
2025-09-09 18:00:40.033424+00:00 SNAPSHOT START 400
2025-09-09 18:00:48.656545+00:00 SNAPSHOT END 400 0:00:08.623121 ELAPSED
2025-09-09 18:00:52.716552+00:00 FINISH BEST-EFFORT WAL CATCHUP 400 

In [52]:
snapshots

version,start,end,wal_catchup,switch_time,switch_version
i64,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",i64
100,2025-09-09 17:58:56.671489 UTC,2025-09-09 17:58:59.621843 UTC,2025-09-09 17:59:01.156508 UTC,2025-09-09 17:59:02.687640 UTC,112
200,2025-09-09 17:59:28.413426 UTC,2025-09-09 17:59:33.319810 UTC,2025-09-09 17:59:36.012194 UTC,2025-09-09 17:59:38.728401 UTC,221
300,2025-09-09 18:00:02.983921 UTC,2025-09-09 18:00:10.125621 UTC,2025-09-09 18:00:13.589576 UTC,2025-09-09 18:00:16.868921 UTC,326
400,2025-09-09 18:00:40.033424 UTC,2025-09-09 18:00:48.656545 UTC,2025-09-09 18:00:52.716552 UTC,2025-09-09 18:00:56.345947 UTC,429
500,2025-09-09 18:01:18.629766 UTC,2025-09-09 18:01:28.799400 UTC,2025-09-09 18:01:34.219341 UTC,2025-09-09 18:01:38.561582 UTC,537
…,…,…,…,…,…
2600,2025-09-09 18:31:34.196718 UTC,2025-09-09 18:32:32.573502 UTC,2025-09-09 18:32:35.316099 UTC,2025-09-09 18:33:14.844786 UTC,2608
2700,2025-09-09 18:34:27.279331 UTC,2025-09-09 18:35:24.491508 UTC,2025-09-09 18:35:26.415380 UTC,2025-09-09 18:35:40.318545 UTC,2706
2800,2025-09-09 18:36:57.098333 UTC,2025-09-09 18:37:59.141726 UTC,2025-09-09 18:38:01.168674 UTC,2025-09-09 18:38:55.145307 UTC,2806


In [54]:
snapshots.select("version", pl.col("switch_time") - pl.col("start"))

version,switch_time
i64,duration[μs]
100,6s 16151µs
200,10s 314975µs
300,13s 885ms
400,16s 312523µs
500,19s 931816µs
…,…
2600,1m 40s 648068µs
2700,1m 13s 39214µs
2800,1m 58s 46974µs
